# Pandas (2)

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

2. Посчитайте количество уникальных цифр, которые используются каждой строке в столбце volume.

3. Для каждой строки рассчитайте разность между значениями high и low, если индекс столбца нечетный, и разность между close и high в противном случае.

4. Рассчитайте суммарный объем торгов для для одинаковых значений тикеров.

5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

## Лабораторная работа №2.2

__Данная работа является продолжением ЛР №2. Для начала работы загрузите таблицы (см. задание 1.1)__

In [5]:
import pandas as pd
import numpy as np

In [195]:
recipes = pd.read_csv('recipes_sample.csv',sep = ',', parse_dates = ['submitted'])
reviews = pd.read_csv('reviews_sample.csv',sep = ',')


In [7]:
recipes['minutes']

0         90
1         10
2         30
3         45
4         25
        ... 
29995     80
29996    240
29997     75
29998     60
29999     29
Name: minutes, Length: 30000, dtype: int64

In [8]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


### Применение функций к pd.Series и pd.DataFrame

4.1 Напишите функцию, которая переводит минуты в формат "XhYm". Примените эту функцию к столбцу `minutes` таблицы `recipes` (без перезаписи исходного столбца) при помощи метода `map`.

In [9]:
def m_to_hm(m: int) -> str:
    return "{}h{}m".format(m//60,m - (m//60)*60)

assert m_to_hm(90) == "1h30m"
assert m_to_hm(10) == "0h10m"


In [10]:
recipes['minutes'].map(m_to_hm)

0        1h30m
1        0h10m
2        0h30m
3        0h45m
4        0h25m
         ...  
29995    1h20m
29996     4h0m
29997    1h15m
29998     1h0m
29999    0h29m
Name: minutes, Length: 30000, dtype: object

In [7]:
# recipes

In [8]:
# new_table = recipes.select_dtypes(include='object')
# new_table.str.capitalize()

In [11]:
new_table = recipes.select_dtypes(include = ['object'])
new_table.applymap(lambda x: str(x).capitalize())

,name,description
0,George s at the cove black bean soup,An original recipe created by chef scott meska...
1,Healthy for them yogurt popsicles,My children and their friends ask for my homem...
2,I can t believe it s spinach,"These were so go, it surprised even me."
3,Italian gut busters,My sister-in-law made these for us at a family...
4,Love is in the air beef fondue sauces,I think a fondue is a very romantic casual din...
...,...,...
29995,Zurie s holey rustic olive and cheddar bread,This is based on a french recipe but i changed...
29996,Zwetschgenkuchen bavarian plum cake,"This is a traditional fresh plum cake, thought..."
29997,Zwiebelkuchen southwest german onion cake,This is a traditional late summer early fall s...
29998,Zydeco soup,This is a delicious soup that i originally fou...


In [12]:
recipes.dtypes

name                      object
id                         int64
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

4.3 Напишите функцию, которая принимает на вход серию `pd.Series` и для серий, содержащих текстовые данные, возвращает максимальную длину строк в ней, а для числовых серий возвращает минимальный элемент в этой серии. Примените данную функцию к каждому столбцу таблицы `recipes` при помощи метода `apply`.

In [13]:
recipes_1 = recipes
recipes_1['submitted'] = recipes_1['submitted'].astype('object')
recipes_1.dtypes

name               object
id                  int64
minutes             int64
contributor_id      int64
submitted          object
n_steps           float64
description        object
n_ingredients     float64
dtype: object

In [18]:
def get_stats(x: pd.Series) -> int:
    if x.dtypes == 'object':
        return int(max(x.str.len()))
    elif x.dtypes == 'int64' or x.dtypes == 'float64':
        x = x.dropna()
        return int(min(x))

assert get_stats(pd.Series(['a', 'bbbb', 'ccc'])) == 4
assert get_stats(pd.Series([1, 3, 2])) == 1

In [19]:
recipes_1.apply(get_stats, axis = 0)

ValueError: cannot convert float NaN to integer

### Группировки таблиц `pd.DataFrame`

5.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [288]:
# recipes[recipes['contributor_id'] == int(recipes[['contributor_id','name']].groupby(['contributor_id']).count().sort_values(by = ['name']).idxmax())] 
# # max_series = new_series.max()
# # new_series['name'].idxmax()

In [20]:
max_series = recipes[['contributor_id','name']].groupby(['contributor_id']).count().sort_values(by = ['name'])
print("contributor_id = {} | count = {}".format(int(max_series.idxmax()), int(max_series.max())))

contributor_id = 89831 | count = 421


5.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [124]:
# recipes[['description','id']].groupby('id').count()['description'].value_counts()

In [125]:
reviews[['recipe_id','rating']].groupby(['recipe_id']).mean()

,rating
recipe_id,
48,1.000000
55,4.750000
66,4.944444
91,4.750000
94,5.000000
...,...
536547,5.000000
536610,0.000000
536728,4.000000


In [126]:
reviews[reviews['review'].isna()]

,Unnamed: 0,user_id,recipe_id,date,rating,review
8881,56957,2001567544,9054,2017-06-03,5,NaN
12129,273481,2002120930,167202,2018-04-18,4,NaN
18578,56955,2001508809,9054,2017-04-26,5,NaN
21913,783599,2001959957,335709,2018-01-28,5,NaN
33232,1089381,2002299388,15553,2018-10-13,4,NaN
42783,266359,2002060663,107786,2018-03-18,5,NaN
42855,1078013,2001346906,522925,2017-01-11,4,NaN
43551,841853,1800574364,391660,2018-08-22,5,NaN
53998,1020267,2002005298,245231,2018-02-22,5,NaN
55100,56966,2001826405,9054,2017-11-26,5,NaN


5.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [287]:
#recipes[recipes['submitted'].dt.year == 2012].count() #это была проста проверка

In [128]:
recipes[['id']].groupby(recipes['submitted'].dt.year).count()

,id
submitted,
1999,275
2000,104
2001,589
2002,2644
2003,2334
2004,2153
2005,3130
2006,3473
2007,4429


5.4 Напишите функцию, которая принимает на вход таблицу (аналогичную `recipes` по набору столбцов), и возвращает `True` в том случае, если в столбце `minutes` присутствуют только значения, меньшие либо равные 10. Сгруппируйте таблицу `recipes` по полю `contributor_id` и для каждого участника выясните, справедливо ли, что все его рецепты занимают не более 10 минут.

In [166]:
def has_only_fast_recipes(x: pd.DataFrame) -> bool:
    return x['minutes'].map(lambda x: x <=10).all()

assert not has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 90, 1: 10},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)
assert has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 7, 1: 5},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)
has_only_fast_recipes(recipes)

False

In [167]:
recip = recipes.groupby('contributor_id').apply(has_only_fast_recipes)
recip

contributor_id
1530          False
1533          False
1534          False
1535          False
1538          False
              ...  
2001968497    False
2002059754    False
2002234079    False
2002234259    False
2002247884    False
Length: 8404, dtype: bool

In [168]:
# recipes[recipes['minutes'] <= 10]

In [169]:
# new_tabl.groupby(['contributor_id']).sum()

### Объединение таблиц `pd.DataFrame`

6.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [97]:
rec_des = recipes[recipes['description'].isna() == False] # df где, нет пустых отзывов
rec_des[['id','name']]

,id,name
0,44123,george s at the cove black bean soup
1,67664,healthy for them yogurt popsicles
2,38798,i can t believe it s spinach
3,35173,italian gut busters
4,84797,love is in the air beef fondue sauces
...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread
29996,386977,zwetschgenkuchen bavarian plum cake
29997,103312,zwiebelkuchen southwest german onion cake
29998,486161,zydeco soup


In [136]:
rev_des = reviews[['user_id','rating']]
rev_des

,user_id,rating
0,21752,5
1,431813,5
2,400708,4
3,2001852463,5
4,95810,5
...,...,...
126691,1270706,4
126692,2282344,0
126693,689540,5
126694,2000242659,5


In [138]:
pd.concat([rec_des[['id','name']], rev_des],ignore_index=True)

,id,name,user_id,rating
0,44123.0,george s at the cove black bean soup,NaN,NaN
1,67664.0,healthy for them yogurt popsicles,NaN,NaN
2,38798.0,i can t believe it s spinach,NaN,NaN
3,35173.0,italian gut busters,NaN,NaN
4,84797.0,love is in the air beef fondue sauces,NaN,NaN
...,...,...,...,...
156068,NaN,NaN,1.270706e+06,4.0
156069,NaN,NaN,2.282344e+06,0.0
156070,NaN,NaN,6.895400e+05,5.0
156071,NaN,NaN,2.000243e+09,5.0


In [234]:
six_1 = rev_des.merge(rec_des[['id','name']], left_on = 'user_id', right_on = 'id')
six_1

,user_id,rating,id,name
0,21752,5,21752,cream of chicken potato bake
1,21752,5,21752,cream of chicken potato bake
2,21752,5,21752,cream of chicken potato bake
3,21752,3,21752,cream of chicken potato bake
4,21752,5,21752,cream of chicken potato bake
...,...,...,...,...
5716,55647,4,55647,tomato herb dressing
5717,57412,5,57412,cucumber red pepper raita
5718,254057,0,254057,chocolate cookarolis fried ravioli cookies
5719,205227,5,205227,pineapple chicken and sweet potatoes crock pot


6.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [220]:
# tabl = reviews[['recipe_id','review']].groupby(['recipe_id']).count()
# tabl.append(recipes['name'])

6.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [219]:
reviews[['date','rating']].groupby(pd.to_datetime(reviews['date']).dt.year).mean()

,rating
date,
2000,3.384615
2001,4.134426
2002,4.481556
2003,4.526940
2004,4.546548
2005,4.563680
2006,4.603673
2007,4.547077
2008,4.484635


In [233]:
d = reviews[['date','rating']].groupby(pd.to_datetime(reviews['date']).dt.year).mean()
d[float(d.min()) == d['rating']]

,rating
date,
2017,3.353042


### Сохранение таблиц `pd.DataFrame`

7.1 Отсортируйте результат выполнения задания 6.1 в порядке убывания величины столбца `id` и сохраните результаты в csv файл. 

In [237]:
six_1.sort_values('user_id')[::-1]

,user_id,rating,id,name
5457,536728,5,536728,gluten free vegemite
5687,534549,5,534549,mimi s date squares gluten free
5720,530536,4,530536,spanakopita spinach pie by ina garten
5486,522240,5,522240,spicy garlic thai shrimp and sauteed kale
5488,522240,2,522240,spicy garlic thai shrimp and sauteed kale
...,...,...,...,...
5359,4753,5,4753,corn casserole iii
5360,4753,4,4753,corn casserole iii
5361,4753,5,4753,corn casserole iii
5362,4753,5,4753,corn casserole iii


7.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 6.1 и 6.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 6.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 6.2.

In [238]:
with pd.ExcelWriter('path_to_file.xlsx') as writer:
    six_1.to_excel(writer, sheet_name='Рецепты с оценками')
    